## IMPORTANT

#### Due a bug in the mini version of gpt2 I faced some issues with generating the output. This notebook is just for experimentation. The notebook used for the project is rag_llama2.ipynb

In [1]:
import pandas as pd

import warnings

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('../data/gold/data.csv')
documents = df.to_dict(orient='records')

In [3]:
documents[0]

{'id': 0,
 'section': 'Overview of This Book',
 'text': 'In the first part of this chapter, I’ll walk through the structure of this book. Then, I’ll discuss the various job titles and roles that use ML skills in industry. 1 I’ll also clarify the responsibilities of various job titles, such as data scientist, machine learning engineer, and so on, as this is a common point of confusion for job seekers. These will be illustrated with an ML skills matrix and ML lifecycle that will be referenced throughout the book. The second part of this chapter walks through the interview process, from beginning to end. I’ve mentored candidates who appreciated this overview since online resources often focus on specific pieces of the interview but not how they all connect together and result in an offer. Especially for new graduates 2 and readers coming from different industries, this chapter helps get everyone on the same page as well as clarifies the process. The interconnecting pieces of interviews ar

## Split into chunks/sentence

In [4]:
import spacy

nlp = spacy.load("es_core_news_sm") 

In [5]:
def split_into_sentence_chunks(text, base_id):
    doc = nlp(text)  
    sentences = [sent.text for sent in doc.sents] 

    chunked_texts = []
    current_chunk = []

    for sentence in sentences:
        current_chunk.append(sentence)

        if len(current_chunk) >= 3:  
            chunked_texts.append(' '.join(current_chunk))
            current_chunk = []  

    if current_chunk:
        chunked_texts.append(' '.join(current_chunk))

    chunk_ids = [f"{base_id}_{i + 1}" for i in range(len(chunked_texts))]

    return [{'chunk_id': chunk_id, 'chunk_text': chunk_text, 'text_id': base_id} 
            for chunk_id, chunk_text in zip(chunk_ids, chunked_texts)]

chunked_docs = []
for doc in documents:
    if 'text' in doc and 'text_id' in doc:  
        chunks = split_into_sentence_chunks(doc['text'], doc['text_id'])
        chunked_docs.extend(chunks)  



In [6]:
chunked_docs[0:2]

[{'chunk_id': '86fd49a66d_1',
  'chunk_text': 'In the first part of this chapter, I’ll walk through the structure of this book. Then, I’ll discuss the various job titles and roles that use ML skills in industry. 1 I’ll also clarify the responsibilities of various job titles, such as data scientist, machine learning engineer, and so on, as this is a common point of confusion for job seekers.',
  'text_id': '86fd49a66d'},
 {'chunk_id': '86fd49a66d_2',
  'chunk_text': 'These will be illustrated with an ML skills matrix and ML lifecycle that will be referenced throughout the book. The second part of this chapter walks through the interview process, from beginning to end. I’ve mentored candidates who appreciated this overview since online resources often focus on specific pieces of the interview but not how they all connect together and result in an offer.',
  'text_id': '86fd49a66d'}]

## minsearch

In [8]:
from minsearch import Index


In [22]:
index = Index(
    text_fields=[ "chunk_text"],
    keyword_fields=["text_id", "chunk_id"]
)


index.fit(chunked_docs)

In [26]:
query = 'what is the scope of a data scientist?'

def search(query):
    boost = {}
    # boost = {'text': 3.0, 'section': 0.5}
    
    results = index.search(
        query=query,
        filter_dict = {},
        boost_dict=boost,
        num_results=2
    )

    return results

search_results = search(query)
search_results

[{'chunk_id': '6d22154063_1',
  'chunk_text': 'Here is a nonexhaustive list of job titles for ML (or closely related) roles: • Data scientist • Machine learning engineer • Applied scientist • Software engineer, machine learning • MLOps engineer • Product data scientist • Data analyst • Decision scientist • Research scientist As I discussed “A Brief History of Machine Learning and Data Science Job Titles” on page 3 , each role is responsible for a different part of the ML lifecycle. A job title alone does not convey what the job entails. As a job seeker, be warned: in different companies, completely different titles might end up doing similar jobs!',
  'text_id': '6d22154063'},
 {'chunk_id': '6d22154063_2',
  'chunk_text': 'As illustrated in Figure 1-3 , your ML job title will depend on the company, the team, and which part(s) of the ML lifecycle your role is responsible for. To give specific examples of how job titles can depend on the company or organiza‐ tion that is hiring for the j

## rag

In [27]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 

torch.random.manual_seed(0)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained("distilgpt2")
model.eval()



## Prompt


In [35]:
prompt_template = """
You are an assistant preparing a candidate for a data science interview. 
Based on the provided context, please provide a concise and accurate answer to the following question in plain text format without any additional formatting.

QUESTION: {question}

CONTEXT:
{context}
""".strip()


In [36]:
# def build_context(search_results):
#     context = []
    
#     for result in search_results:
#         text = result.get("chunk_text", "")
        
#         context.append(f"Text: {text}")
    
#     return "\n\n".join(context)

# context = build_context(search_results)
# print(context)


Text: Here is a nonexhaustive list of job titles for ML (or closely related) roles: • Data scientist • Machine learning engineer • Applied scientist • Software engineer, machine learning • MLOps engineer • Product data scientist • Data analyst • Decision scientist • Research scientist As I discussed “A Brief History of Machine Learning and Data Science Job Titles” on page 3 , each role is responsible for a different part of the ML lifecycle. A job title alone does not convey what the job entails. As a job seeker, be warned: in different companies, completely different titles might end up doing similar jobs!

Text: As illustrated in Figure 1-3 , your ML job title will depend on the company, the team, and which part(s) of the ML lifecycle your role is responsible for. To give specific examples of how job titles can depend on the company or organiza‐ tion that is hiring for the job—based on real people I’ve spoken to, job descriptions, and job interviews—the person responsible for trainin

In [37]:
def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        text = doc['chunk_text']
            
        context += f"Text: {text}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

prompt = build_prompt(query, search_results)
print(prompt)

You are an assistant preparing a candidate for a data science interview. 
Based on the provided context, please provide a concise and accurate answer to the following question in plain text format without any additional formatting.

QUESTION: what is the scope of a data scientist?

CONTEXT:
Text: Here is a nonexhaustive list of job titles for ML (or closely related) roles: • Data scientist • Machine learning engineer • Applied scientist • Software engineer, machine learning • MLOps engineer • Product data scientist • Data analyst • Decision scientist • Research scientist As I discussed “A Brief History of Machine Learning and Data Science Job Titles” on page 3 , each role is responsible for a different part of the ML lifecycle. A job title alone does not convey what the job entails. As a job seeker, be warned: in different companies, completely different titles might end up doing similar jobs!

Text: As illustrated in Figure 1-3 , your ML job title will depend on the company, the team,

## llm

In [41]:
def llm(prompt):
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True,  max_length=1000)
    with torch.no_grad():
        outputs = model.generate(**inputs, pad_token_id=tokenizer.eos_token_id, num_return_sequences=1,max_new_tokens=50) 
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


In [42]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)    
    answer = llm(prompt)

    return answer

In [43]:
print(rag(query))

You are an assistant preparing a candidate for a data science interview. 
Based on the provided context, please provide a concise and accurate answer to the following question in plain text format without any additional formatting.

QUESTION: what is the scope of a data scientist?

CONTEXT:
Text: Here is a nonexhaustive list of job titles for ML (or closely related) roles: • Data scientist • Machine learning engineer • Applied scientist • Software engineer, machine learning • MLOps engineer • Product data scientist • Data analyst • Decision scientist • Research scientist As I discussed “A Brief History of Machine Learning and Data Science Job Titles” on page 3 , each role is responsible for a different part of the ML lifecycle. A job title alone does not convey what the job entails. As a job seeker, be warned: in different companies, completely different titles might end up doing similar jobs!

Text: As illustrated in Figure 1-3 , your ML job title will depend on the company, the team,

## Retrieval evaluation

In [14]:
df_questions = pd.read_csv('../data/ground_truth_data.csv')

In [ ]:
df_questions

In [16]:
ground_truth = df_questions.to_dict(orient = 'records')

In [ ]:
ground_truth[0]

In [ ]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [ ]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [ ]:
def minsearch_search(query):
    boost = {}
    # boost = {'text': 3.0, 'section': 0.5}
    
    results = index.search(
        query=query,
        filter_dict = {},
        boost_dict=boost,
        num_results=2
    )

    return results



In [ ]:
from tqdm.auto import tqdm


def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [ ]:
from tqdm.auto import tqdm

In [ ]:
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

In [ ]:
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

## Hyperparams Optimization


In [18]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [ ]:
df_val = df_questions[:100]
df_test = df_questions[100:]

### Search space

In [ ]:
from hyperopt import hp

# Espacio de búsqueda para Hyperopt
space = {
    'temperature': hp.uniform('temperature', 0.5, 1.5),  # Ajuste de temperatura
    'max_new_tokens': hp.choice('max_new_tokens', [20, 50, 100]),  # Límite de tokens generados
}

### Loss function

In [ ]:
def objective(params):
    temperature = params['temperature']
    max_new_tokens = params['max_new_tokens']

    # Configurar el modelo con los parametros de arriba que faltan eh?
    output_tokens = model.generate(
        **inputs, 
        pad_token_id=tokenizer.eos_token_id, num_return_sequences=num_return_sequences,max_new_tokens=max_new_tokens) 
    )
    
    generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True).strip()

    # Evaluar la calidad del texto generado con las métricas definidas
    # Asumamos que tienes una función calculate_mrr que recibe las predicciones y el ground truth
    mrr_score = mrr(generated_text)
    
    # Devuelve la métrica negativa para que Hyperopt lo minimice
    return {'loss': -mrr_score, 'status': 'ok'}


In [ ]:
from hyperopt import fmin, tpe, Trials

# Inicializar el historial de pruebas
trials = Trials()

# Ejecutar la búsqueda
best = fmin(
    fn=objective,  # Función objetivo
    space=space,  # Espacio de búsqueda
    algo=tpe.suggest,  # Algoritmo de optimización
    max_evals=10,  # Número de evaluaciones
    trials=trials  # Historial de pruebas
)

print("Mejores hiperparámetros encontrados:", best)



In [ ]:
gt_val = df_val.dict(orient='records')

In [ ]:
evaluate(gt_val, lambda q: minsearch_search(q['question']))
# para mirar cuanto da con los mejores hyperparam 

In [ ]:
def minsearch_search_optimized(query):
    # boost = {'text': 3.0, 'section': 0.5} AQUI VAN LOS HYPERPARAMS 
    
    results = index.search(
        query=query,
        filter_dict = {},
        boost_dict=boost,
        num_results=2
    )

    return results



In [ ]:
evaluate(ground_truth, lambda q: minsearch_search_optimized(q['question']))
#copiar y pegar en el README
# tb los best boosting params


## RAG Evaluation

In [ ]:
prompt1_template = """
You are an expert evaluator for a Retrieval-Augmented Generation (RAG) system.
Your task is to analyze the relevance of the generated answer compared to the original answer provided.
Based on the relevance and similarity of the generated answer to the original answer, you will classify
it as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Original Answer: {answer_orig}
Generated Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the original
answer and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

prompt2_template = """
You are an expert evaluator for a Retrieval-Augmented Generation (RAG) system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()


In [ ]:
len(ground_truth) 